In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [3]:
df = pd.read_csv('./imdb_all_v2.csv')
# df = dframe[(1920 <= dframe['year']) & (dframe['year'] <= 2021)]
genre_list = []
for i in range(len(df)):
    try:
        genre_list.append(df.iloc[i]['genre'].split(','))
    except:
        genre_list.append(None)
df['genre_list'] = genre_list
df = df.explode('genre_list')
df['genre_list'] = df['genre_list'].apply(lambda x: x.strip())
genre_group = df.groupby('genre_list')

genres = []
number = []
years = []
for genre in genre_group:
    # for year_group in genre[1].groupby('year'):
    for i in range(1920, 2022):
        years.append(i)
        genres.append(genre[0])
        number.append(len(genre[1][genre[1]['year'] == i]))
output = pd.DataFrame(data = [genres, number, years]).T
output.columns = ['genre', 'count', 'year']

In [4]:
trace_dict = {}
all_genre = output['genre'].unique()

for genre in all_genre:
    trace_dict[genre] = go.Scatter(
                            x= output['year'], y = output[output['genre'] == genre]['count'],
                            name = genre,
                            mode = 'none', 
                            stackgroup = 'one', line_shape = 'spline')

In [5]:
fig = go.Figure()
for genre in trace_dict.keys():
    fig.add_trace(trace_dict[genre])

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.show()